# CMSC320 Final Project: Chess Data Analysis
##### *Mohammad Durrani and Pranav Shah*

### Contents
1. [Introduction](##1.-Introduction)
2. [Required Tools](##2.-Required-Tools)
3. [About The Data](##3.-About-The-Data)
4. [Data Collection: Scraping](##4.-Data-Collection:-Scraping)
5. [Data Processing: Cleaning](##5.-Data-Processing:-Cleaning)
6. [Exploratory Data Analysis](##6.-Exploratory-Data-Analysis)
7. [Future Work and Conclusion](##7.-Future-Work-and-Conclusion)
8. [References and Additional Resources](##8.-References-and-Additional-Resources)

## 1. Introduction

## 2. Required Libraries

In [9]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install chess
!{sys.executable} -m pip install category-encoders


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



                                              0.0/81.9 kB ? eta -:--:--
     ---------------------------------------- 81.9/81.9 kB 2.3 MB/s eta 0:00:00
                                              0.0/9.9 MB ? eta -:--:--
                                              0.2/9.9 MB 5.3 MB/s eta 0:00:02
     -                                        0.3/9.9 MB 3.5 MB/s eta 0:00:03
     --                                       0.6/9.9 MB 4.6 MB/s eta 0:00:03
     ---                                      1.0/9.9 MB 5.5 MB/s eta 0:00:02
     -----                                    1.4/9.9 MB 6.1 MB/s eta 0:00:02
     -------                                  1.8/9.9 MB 6.8 MB/s eta 0:00:02
     ---------                                2.3/9.9 MB 7.4 MB/s eta 0:00:02
     -----------                              2.8/9.9 MB 7.9 MB/s eta 0:00:01
     -------------                            3.3/9.9 MB 8.1 MB/s eta 0:00:01
     --------------                           3.6/9.9 MB 7.9 MB/s eta 0:00


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## 3. About The Data

## 4. Data Collection: Scraping

In [ ]:
import chess.pgn
import pandas as pd
import glob
import numpy as np

#pgnFilePath =  r"\\wsl.localhost\Ubuntu\home\mdurrani\lichess_db_standard_rated_2024-03.pgn"
pgnFilePath = r'./data/*.pgn'
outputFilePath = "./evaluations.csv"
totalGameCount = 0
totalEvalGameCount = 0
fileList = [] #glob.glob(pgnFilePath) # modify this line for the directory that has all the pgn files 
rows_list = [] # list of dictionaries for each game 
for file in fileList:
    pgn = open(fileList[0])
    game = chess.pgn.read_game(pgn)
    while game != None:
        totalGameCount += 1
        variations = game.variations # list of either the eval or clock score
        if(totalGameCount % 2000 == 0): # print every 2000 games
            print("Total Game Count: ", totalGameCount)
            print("Eval Game Count: ", totalEvalGameCount)
        if(totalGameCount % 1000000 == 0):
            df = pd.DataFrame(rows_list) # add everything to the dataframe 
            df.to_csv(outputFilePath)
        if(len(variations) > 0 and 'eval' in variations[0].comment): # if this game was evaluated by a computer, add it
            totalEvalGameCount+=1
            h = game.headers
                
            # adding a dictionary is faster than appending to a dataframe 
            rows_list.append({
                "UTCDate": h.get("UTCDate", np.NaN),
                "UTCTime": h.get("UTCTime", np.NaN),
                'WhiteElo': h.get('WhiteElo', np.NaN),
                'BlackElo': h.get('BlackElo', np.NaN),
                "Opening": h.get("Opening", np.NaN),
                "ECO": h.get("ECO", np.NaN),
                'Result': h.get('Result', np.NaN),
                "Termination": h.get("Termination", np.NaN),
                "Variations": str(variations[0]) if variations else np.NaN,
                'WhiteRatingDiff': h.get('WhiteRatingDiff', np.NaN),
                'BlackRatingDiff': h.get("BlackRatingDiff", np.NaN)
            })

        # Iterator reading next game
        game = chess.pgn.read_game(pgn)          

## 5. Data Processing: Cleaning

## 6. Exploratory Data Analysis

In [21]:
import pandas as pd 
import re
from sklearn import preprocessing
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

df= pd.read_csv("miniEvaluations.csv")
# Get white mistake differential from a game
def getMistakeDifferential(variation):
    # Find all evaluations
    evalText = re.findall(r'%eval -?\d.\d*', variation)
    # Truncate text and get float eval value
    evalList = []
    for eval in evalText:
        evalList.append(float(eval.split(" ")[1]))

    # Find the mistake differential for white
    evalDifference = 0
    numberOfWhiteMistakes = 0
    numberOfBlackMistakes = 0
    for i in range(len(evalList)):
        if i != 0:
            evalDifference = evalList[i] - evalList[i-1]
        if abs(evalDifference) > 1:
            if i % 2 == 0:
                numberOfWhiteMistakes += 1
            else:
                numberOfBlackMistakes += 1
    whiteMistakeDifferential = numberOfWhiteMistakes - numberOfBlackMistakes
    return whiteMistakeDifferential

# Get white time differential from a game
def getTimeDifferential(variation):
    # Find all clock text
    clockText = re.findall(r'%clk \d:\d{2}:\d{2}', variation)
    # Truncate text and get clock data in seconds
    clockList = []
    for clock in clockText:
        time = clock.split(" ")[1]
        hr = int(time.split(":")[0])
        min = int(time.split(":")[1])
        sec = int(time.split(":")[2])
        totalSeconds = hr * 3600 + min * 60 + sec
        clockList.append(totalSeconds)

    # Get clock differential
    whiteTimeElapsed = 0
    blackTimeElapsed = 0
    # If less than 2 moves, then don't worry about calculating time elapsed
    if len(clockList) >= 2:
        whiteBeginClock = clockList[0]
        blackBeginClock = clockList[1]
        if len(clockList) % 2 == 0:
            whiteEndClock = clockList[-2]
            blackEndClock = clockList[-1]
        else:
            whiteEndClock = clockList[-1]
            blackEndClock = clockList[-2]
        whiteTimeElapsed = whiteBeginClock - whiteEndClock
        blackTimeElapsed = blackBeginClock - blackEndClock

    whiteTimeDifferential = whiteTimeElapsed - blackTimeElapsed

    return whiteTimeDifferential

# Turn string result into a number result from white's perspective   
def getResultForWhite(result):
    if result == "0-1":
        return 0
    elif result == "1-0":
        return 1
    else:
        return 0.5
    
df["whiteMistakeDifferential"] = df["Variations"].apply(getMistakeDifferential)

df["whiteTimeDifferential"] = df["Variations"].apply(getTimeDifferential)

df["whiteEloDifferential"] = df["WhiteElo"] - df["BlackElo"]

df["whiteResult"] = df["Result"].apply(getResultForWhite)

trainingDf = df.loc[:, ["whiteMistakeDifferential", "whiteTimeDifferential", "whiteEloDifferential", "whiteResult"]]

label_encoder = ce.BinaryEncoder()
bin_enc_df = label_encoder.fit_transform(df["ECO"])
trainingDf = trainingDf.join(bin_enc_df)



X = trainingDf
y = df['whiteResult']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

,whiteMistakeDifferential,whiteTimeDifferential,whiteEloDifferential,whiteResult,ECO_0,ECO_1,ECO_2,ECO_3,ECO_4,ECO_5,ECO_6,ECO_7,ECO_8
0,2,26,97,0.0,0,0,0,0,0,0,0,0,1
1,-1,-33,-128,1.0,0,0,0,0,0,0,0,1,0
2,-3,-495,406,1.0,0,0,0,0,0,0,0,1,1
3,1,-527,342,1.0,0,0,0,0,0,0,1,0,0
4,-3,121,441,1.0,0,0,0,0,0,0,1,0,1


## 7. Future Work and Considerations

## 8. References and Additional Resources